In [28]:
# Import Stuff
import os
import keras
import scipy.misc
import numpy as np
import tensorflow as tf
from skimage import color
from keras.optimizers import SGD
from keras import metrics
from keras.models import load_model
from skimage.transform import resize
from keras.preprocessing import image
from keras.callbacks import TensorBoard
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers.core import RepeatVector, Permute
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, InputLayer, Reshape, merge, concatenate

In [19]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2
import tensorflow as tf
inception = InceptionResNetV2(weights=None,include_top=True)
inception.load_weights('inception_resnet_v2_weights_tf_dim_ordering_tf_kernels.h5')
inception.graph = tf.get_default_graph()

In [5]:
model = load_model('eigth_8k_images_200.h5')

In [6]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 128, 64) 640         input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 128, 128, 128 73856       conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 64, 64, 128)  147584      conv2d_2[0][0]                   
__________________________________________________________________________________________________
conv2d_4 (

In [7]:
# Get images
X = []
count=0
for filename in os.listdir('Train_Iterative/Train_7/'):
    if filename.endswith('.jpg') or filename.endswith('.jpeg'):
        X.append(image.img_to_array(image.load_img('Train_Iterative/Train_7/'+filename)))
        count+=1;
        if count%128==0:
          print(count)
X = np.array(X, dtype=float)
Xtrain = X/255.0

128
256
384
512
640
768
896


In [8]:
embed_input = Input(shape=(1000,))

In [9]:
# Encoder
encoder_input = Input(shape=(256, 256, 1,))
encoder_output = Conv2D(64, (3,3), activation='relu', padding='same', strides=2)(encoder_input)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)

In [10]:
# Fusion
fusion_output = RepeatVector(32 * 32)(embed_input)
fusion_output = Reshape(([32, 32, 1000]))(fusion_output)
fusion_output = concatenate([encoder_output, fusion_output], axis=3)
fusion_output = Conv2D(256, (1, 1), activation='relu', padding='same')(fusion_output)

In [11]:
# Decoder
decoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(fusion_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(64, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(32, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(16, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(2, (3, 3), activation='tanh', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)

In [12]:
model = Model(inputs=[encoder_input, embed_input], outputs=decoder_output)

In [13]:
# Create embedding
def create_inception_embedding(grayscaled_rgb):
    grayscaled_rgb_resized = []
    for i in grayscaled_rgb:
        i = resize(i, (299, 299, 3), mode='constant')
        grayscaled_rgb_resized.append(i)
    grayscaled_rgb_resized = np.array(grayscaled_rgb_resized)
    grayscaled_rgb_resized = preprocess_input(grayscaled_rgb_resized)
    with inception.graph.as_default():
        embed = inception.predict(grayscaled_rgb_resized)
    return embed

In [14]:
# Image transformer
datagen = image.ImageDataGenerator(shear_range=0.4,
                            zoom_range=0.4,
                            rotation_range=40,
                            horizontal_flip=True)

In [15]:
# Generate training data
batch_size = 64

In [ ]:
import csv
def image_a_b_gen(batch_size):
    for batch in datagen.flow(Xtrain,batch_size=batch_size):
        grayscaled_rgb = color.gray2rgb(color.rgb2gray(batch))
        embed = create_inception_embedding(grayscaled_rgb)
        lab_batch = color.rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        X_batch = X_batch.reshape(X_batch.shape+(1,))
        with open('hh.csv', 'w') as writeFile:
            writer = csv.writer(writeFile)
            writer.writerows(X_batch)
        break
        Y_batch = lab_batch[:,:,:,1:] / 128
        print ([X_batch, create_inception_embedding(grayscaled_rgb)], Y_batch)
a=image_a_b_gen(64)



In [20]:
#Train model
tensorboard = TensorBoard(log_dir="Output_complete/")
model.compile(optimizer='adam', loss='mse')
alpha = model.fit_generator(image_a_b_gen(batch_size), callbacks= [tensorboard], epochs=200, steps_per_epoch=1)

Epoch 1/200
[[[[53.09284099]
   [52.74973745]
   [52.30518376]
   ...
   [29.81962718]
   [29.6274802 ]
   [30.32387672]]

  [[53.46915998]
   [53.24735545]
   [53.02542542]
   ...
   [29.56064258]
   [30.1452826 ]
   [30.32387672]]

  [[53.8451229 ]
   [53.62352628]
   [53.40180907]
   ...
   [29.92648262]
   [30.32387672]
   [30.32387672]]

  ...

  [[87.22435317]
   [86.88752669]
   [86.54798362]
   ...
   [47.04296245]
   [44.52989035]
   [45.63706728]]

  [[86.98586937]
   [86.66191936]
   [85.87025064]
   ...
   [49.34423268]
   [48.30273439]
   [46.28174748]]

  [[86.7572915 ]
   [86.15683614]
   [85.82830129]
   ...
   [44.93890285]
   [48.6248658 ]
   [49.02766153]]]


 [[[ 5.23366507]
   [ 7.84095357]
   [11.17011681]
   ...
   [66.3604791 ]
   [66.881991  ]
   [67.31008987]]

  [[ 4.92946699]
   [ 6.3304463 ]
   [ 9.59735239]
   ...
   [67.26149236]
   [67.81492035]
   [68.80068467]]

  [[ 4.96101485]
   [ 5.21116405]
   [ 7.80298165]
   ...
   [68.68641337]
   [69.12494964]

StopIteration: 

In [14]:
model.save('seventh_7k_images_200.h5')

In [16]:
# Get images
X = []
count=0
for filename in os.listdir('Train_Iterative/Train_8/'):
    if filename.endswith('.jpg') or filename.endswith('.jpeg'):
        X.append(image.img_to_array(image.load_img('Train_Iterative/Train_8/'+filename)))
        count+=1;
        if count%128==0:
          print(count)
X = np.array(X, dtype=float)
Xtrain = X/255.0

128
256
384
512
640
768
896


In [17]:
#Train model
tensorboard = TensorBoard(log_dir="Output_complete/")
# model.compile(optimizer='adam', loss='mse')
alpha = model.fit_generator(image_a_b_gen(batch_size), callbacks= [tensorboard], epochs=200, steps_per_epoch=1)

Epoch 1/200
1/1 [==============================] - 162s 162s/step - loss: 0.0120
Epoch 2/200
1/1 [==============================] - 132s 132s/step - loss: 0.0075
Epoch 3/200
1/1 [==============================] - 155s 155s/step - loss: 0.0094
Epoch 4/200
1/1 [==============================] - 152s 152s/step - loss: 0.0080
Epoch 5/200
1/1 [==============================] - 114s 114s/step - loss: 0.0112
Epoch 6/200
1/1 [==============================] - 97s 97s/step - loss: 0.0073
Epoch 7/200
1/1 [==============================] - 96s 96s/step - loss: 0.0083
Epoch 8/200
1/1 [==============================] - 108s 108s/step - loss: 0.0125
Epoch 9/200
1/1 [==============================] - 120s 120s/step - loss: 0.0087
Epoch 10/200
1/1 [==============================] - 118s 118s/step - loss: 0.0087
Epoch 11/200
1/1 [==============================] - 116s 116s/step - loss: 0.0085
Epoch 12/200
1/1 [==============================] - 116s 116s/step - loss: 0.0067
Epoch 13/200
1/1 [===========

In [1]:
model.save('eigth_8k_images_200.h5')

NameError: name 'model' is not defined

In [38]:
#Make a prediction on the unseen images
color_me = []
for filename in os.listdir('Test_full/'):
    if filename.endswith('.jpg') or filename.endswith('.jpeg'):
        color_me.append(image.img_to_array(image.load_img('Test_full/'+filename)))

color_me = np.array(color_me, dtype=float)
# color_me = color_me.reshape(1, 256, 256, 1)
color_me = color_me/255.0
color_me = color.gray2rgb(color.rgb2gray(color_me))
color_me_embed = create_inception_embedding(color_me)
color_me = color.rgb2lab(color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))

In [39]:
# Test model
output = model.predict([color_me, color_me_embed])
output = output * 128

In [40]:
# Output colorizations

import scipy.misc

for i in range(len(output)):
    cur = np.zeros((256, 256, 3))
    cur[:,:,0] = color_me[i][:,:,0]
    cur[:,:,1:] = output[i]
    scipy.misc.imsave("Output_complete/img_"+str(i)+".png", color.lab2rgb(cur))

/Users/shikharagnihotri/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:9: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
